In [6]:
!pip install pandas pymongo dash jupyter-dash

In [7]:
# Stacey Griggs
# 7/20/2025
# CS 499
# Setup the Jupyter version of Dash
from dash import Dash
import dash_leaflet as dl         # For map visualizations
from dash import dcc              # For interactive components (dropdowns, radio buttons, graphs)
from dash import html             # For HTML elements in layout
import plotly.express as px       # For creating charts and graphs
from dash import dash_table       # For displaying tabular data interactively
from dash.dependencies import Input, Output, State     # For connecting interacitivity between components
import base64                     # For encoding images
import pandas as pd               # For handling table data
import math

# Import CRUD Python module and class to talk to the database
from animal_shelter import AnimalShelter

# Set up database access credentials 
#username = "aacuser"
#password = "SNHU1234"



#########################
# Database Load
#########################
db = AnimalShelter()
df = pd.DataFrame.from_records(db.read({}))   # Get all records from the database and put them in pandas table - DataFrame
# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)


# Pie chart columns: only use if they exist in the DataFrame
pie_chart_columns = [col for col in ['animal_type', 'breed', 'sex_upon_outcome', 
                                     'age_upon_outcome', 'outcome_type', 'color'] if col in df.columns]

# Haversind distance calculator: Math Function to calculate distance of shelter from cities
def haversine(lat1, lon1, lat2, lon2):
    R = 3958.8  # miles
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return R * (2 * math.atan2(math.sqrt(a), math.sqrt(1 - a)))



# Debug: Test to show first few rows in the terminal for debugging
# print(df.head())    


# Debug: Test used to print info about the data - uncomment to use
# print(len(df.to_dict(orient='records')))

# Debug: Test Column Names
#print(df.columns)


#########################
# Dashboard Layout / View
#########################

# Make the Dash app
app = Dash(__name__)

# Load and encode the logo image so it can be displayed in Dash
image_filename = 'Grazioso Salvare Logo.png'
try:
    with open(image_filename, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read())
except FileNotFoundError:
    encoded_image = b''


# Put all the dashboard parts together
app.layout = html.Div([

# Container Top
    # Section Logo + App Title
    #Display logo in left of title
    #Displaly dashboard title - centere + bold
    html.Div([
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()), 
            style={
                'height': '125px',
                'width': '100px',
                'borderRadius': '50%',            # Makes it a circle
                'border': '4px solid #C70039',        # Red border
                'backgroundColor': 'white',       # White background inside the circle
                'padding': '5px',                 # Add space inside the circle
                'boxShadow': '0 2px 6px rgba(0, 0, 0, 0.2)',  # Soft shadow for depth
                'objectFit': 'cover',             # Scales image 
                'margin-right': '20px'
            }
        ),
        html.H1(
            'Stacey Griggs CS-340 Dashboard', 
            style={'margin': '0',
                  'fontWeight': 'bold',
                  'fontSize': '2.5em'
            }
        )
    ], style= {
        'display': 'flex',
        'alignItems': 'center',
        'justifyContent': 'center',
        'padding': '10px',
        'backgroundColor': '#f0f4f8',
        'boxShadow': '0 4px 6px rgba(0, 0, 0, 0.1)'
    }),        
        
    # Container - Main Content = Split into Left Column and Right Column
    html.Div([        

        # Column Left
        html.Div([
            html.H2("Animal Rescue Data Table", style={'textAlign': 'center'}),

            # Row: Radio + Dropdown + Distance Box
            html.Div([
                
                # Section: Left - Radio Filter: Rescue Type
                html.Div([
                    html.H4("Filter by Rescue Type"),
                    dcc.RadioItems(
                        id='filter-type',
                        options=[
                            {'label': 'Reset', 'value': 'RESET'},
                            {'label': 'Water Rescue', 'value': 'WATER'},
                            {'label': 'Mountain or Wilderness Rescue', 'value': 'MOUNTAIN_OR_WILDERNESS'},
                            {'label': 'Disaster or Individual Tracking', 'value': 'DISASTER_OR_INDIVIDUAL_TRACKING'}
                        ],
                        value='RESET'
                    )
                ], style={'marginRight': '40px'}),
            
                # Section: Middle - Dropdown: Sort by
                html.Div([ 
                    html.H4("Sort by:"),
                    dcc.Dropdown(
                        id='sort-dropdown',
                        options=[
                            {'label': 'Age', 'value': 'age_upon_outcome_in_weeks'},
                            {'label': 'Breed', 'value': 'breed'},
                            {'label': 'Color', 'value': 'color'},
                            {'label': 'Outcome Type', 'value': 'outcome_type'}
                        ],
                        placeholder='Sort by',
                        clearable=True,
                        style={'width': '200px'}
                    ),
                ], style={'marginRight': '40px'}),
                
                # Section: Right - Distance from Austin
                html.Div([
                    html.H4("Distance from Austin (mi):"),
                    html.Div(id='distance-box', style={
                        'padding': '10px',
                        'backgroundColor': '#f2f2f2',
                        'border': '1px solid #ccc',
                        'width': '150px'
                    })
                ])
            ], style={'display': 'flex', 'alignItems': 'flex-start', 'marginBottom': '20px'}),
     
            # Section: Data Table
            dash_table.DataTable(
                id='datatable-id',
                columns = [{"name": i, "id": i, "deletable": False, "selectable": True}
                    for i in df.columns],
                data=df.to_dict('records'),   # Show all data at startup
                editable=False,               # No editing in the table
                filter_action="native",       # Built-in filtering 
                sort_action="native",         # Built-in sorting
                sort_mode="multi",            # Can sort by more than one column
                column_selectable="multi",    # Can select more than one column
                row_selectable="multi",       # Can select more than one row
                row_deletable=False,          # Cannot delete rows
                selected_columns=[],          # No columns selected at first
                selected_rows=[],             # No rows selected at first
                page_action="native",         # Built-in paging
                page_current=0,               # Start on the first page
                page_size=10,                 # Only show 10 rows per page
                style_cell={
                    'minWidth': '100px', 'maxWidth': '180px',
                    'whiteSpace': 'normal',
                    'textAlign': 'left'
                },
                style_table={
                    'overflowX': 'auto',
                    'overflowY': 'auto',
                    'height': '500px',
                    'marginTop': '10px'} # Allow horizontal scrolling
            )
        ], style={
            'flex': '1',
            'padding': '20px',
            'minWidth': '350px'
        }),

# Column Right
        # Row: Dropdown + Pie Chart + Map = Stacked Vertical
        html.Div([
            html.H2("Pie Chart View", style={'textAlign': 'center', 'marginBottom': '5px'}),

            # Section: Top - Dropdown
            html.Div([          
                html.H4("Make a Selection", style={'marginBottom': '5px', 'marginTop': '0px'}),
                dcc.Dropdown(
                    id='pie_dropdown',
                    options=[{'label': col, 'value': col} for col in pie_chart_columns],
                    value=pie_chart_columns[0] if pie_chart_columns else None,
                    clearable=False,
                     style={'width': '100%', 'marginBottom': '5px'}
                ),
                
                # Section: Middle - Pie Chart
                html.Div(id='graph-id', style={'height': '300px', 'width': '100%'})
            ], style={
                'display': 'flex',
                'flexDirection': 'column',
                'padding':'5px'
            }),
            
            # Section: Bottom - Map
            html.H2("Map View", style={'textAlign': 'center', 'marginTop': '0px'}),
            html.Div(
                id='map-id',
                style={'height': '350px', 'width': '100%', 'marginTop': '0px'}
            )
        ], style={
            'flex': '1',
            'padding': '20px',
            'minWidth': '250px',
            'maxWidth': '400px',
        })
    ], style={
        'display': 'flex',
        'width': '100%',
        'backgroundColor': '#7a94b0',  
        'minHeight': '100vh',          
        'padding': '20px'
    })
])
#############################################
# Callbacks = Interaction Between Components / Controller
#############################################

# Callback to update the data table
@app.callback(Output('datatable-id','data'),
              Output('datatable-id','columns'),
              Output('datatable-id','selected_rows'),
              [Input('filter-type', 'value'),
               Input('sort-dropdown', 'value')]
)

def update_dashboard(filter_type, sort_by):
    
    # Filter the data based on selection
    if filter_type == 'RESET':
        df = pd.DataFrame.from_records(db.read({}))   # Show all records
        
    elif filter_type == 'WATER':
        df =  pd.DataFrame(list(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
            "outcome_type": {"$nin": ["Euthanasia"]}
        })))
    elif filter_type == 'MOUNTAIN_OR_WILDERNESS':
        df = pd.DataFrame(list(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", 
                              "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
            "outcome_type": {"$nin": ["Euthanasia"]}
        })))
    
    elif filter_type == 'DISASTER_OR_INDIVIDUAL_TRACKING':
        df = pd.DataFrame(list(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                              "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
            "outcome_type": {"$nin": ["Euthanasia"]}
        })))
        
    else:
        df = pd.DataFrame.from_records(db.read({}))   # Default, show all records

    # Custom Sort by Filters
    if sort_by:
        df = df.sort_values(by=sort_by)

    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
    
    # Prepare the table data and columns for Dash
    data = df.to_dict('records')
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    selected_rows = [0] if len(data) > 0 else []

    return (data, columns, selected_rows)

# Callback to update distance
@app.callback(
    Output('distance-box', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_distance(rows, selected_rows):
    if not selected_rows or rows is None:
        return "Distance: N/A"

    selected_row_index = selected_rows[0]
    try:
        selected_animal = rows[selected_row_index]
        animal_lat = float(selected_animal.get('location_lat', 0))
        animal_lon = float(selected_animal.get('location_long', 0))
    except (IndexError, ValueError, TypeError):
        return "Distance: N/A"

    austin_lat = 30.70993564
    austin_lon = -97.69685334

    distance = haversine(animal_lat, animal_lon, austin_lat, austin_lon)
    return f"{distance:.1f} mi"    
    

# Callback to highlight the selected rows in the table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    if not selected_rows:
        return []
    # Highlight each selected row with a light red color to match theme
    return [{
        'if': {'row_index': i},
        'background_color': '#FAA0A0'
    } for i in selected_rows]

# Callback to update the pie chart when the table or dropdown changes
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', "derived_virtual_data"),
     Input('pie_dropdown', 'value')]
)
def update_graphs(viewData, dropdownValue):
    # If there is no data - show message
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data to display.")]
    dff = pd.DataFrame.from_dict(viewData)
    #Check that the selected column exists
    if dropdownValue not in dff.columns:
        return [html.Div(f"Column '{dropdownValue}' not found.")]
    # Show the pie chart for the selected column
    return [
        dcc.Graph(
            figure = px.pie(
                data_frame=dff, 
                names=dropdownValue,
                title=f"Distribution of {dropdownValue}"
            )
        )
    ]

# Callback to update the map whehn the table or row selection changes
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, selected_rows):
    # Default map center - Austin, TX
    default_center = [30.2672, -97.7431]
    default_zoom = 12

    # If no data or no row selected, show message
    if not viewData or not selected_rows:
        return [html.Div("No map data available.")]
   
    #Get the selected row from data table
    dff = pd.DataFrame.from_dict(viewData)
    row = selected_rows[0]   #Default to first row if nothing is selected

    # Error handling for lat/lon
    try:
        lat = dff.at[row, 'location_lat']
        lon = dff.at[row, 'location_long']
    except KeyError:
        return [html.Div("Latitude/Longitude columns not found.")]

    # If no location, show a message
    if pd.isna(lat) or pd.isna(lon):
        return [html.Div("Selected row has no valid location data.")]

    # Show the map with the animal's location marked
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row, 4]),     # Show some info when hovering
                dl.Popup([
                    html.H1("Animal Name"),       # Show animal name/details in popup
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Run app using externally using Jupyter
#app.run_server(mode='external')

# For Python script use:
app.run(debug=True)
